In [ ]:
import zipfile
import os

def unzip_folder(zip_file_path, destination_folder):
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(destination_folder)
        print("Folder successfully unzipped to:", destination_folder)
    except zipfile.BadZipFile:
        print("Error: Invalid ZIP file.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    zip_file_path = "/content/Labeled CV Dataset.zip"  # Replace with the actual path to your ZIP file
    destination_folder = "/content"  # Replace with the destination folder path

    unzip_folder(zip_file_path, destination_folder)


Folder successfully unzipped to: /content


In [ ]:
!pip install tensorflow
!pip install numpy
!pip install opencv-python


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, Model, optimizers
import numpy as np
import cv2

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, Model, optimizers, regularizers

def create_model():
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

    # Freeze the base model's layers
    base_model.trainable = False

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)

    # Additional hidden layers
    x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)

    # Output layer with softmax activation for multiclass classification
    output = layers.Dense(3, activation='softmax')(x)  # Three classes: currency, barcode, text

    model = Model(inputs=base_model.input, outputs=output)

    return model

if __name__ == "__main__":
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_dir = '/content/Labeled CV Dataset/train'
    test_dir = '/content/Labeled CV Dataset/test'

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    model = create_model()

    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_generator,
              epochs=10,
              steps_per_epoch=len(train_generator),
              validation_data=test_generator,
              validation_steps=len(test_generator))

    _, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
    print(f"Test accuracy: {test_accuracy}")

    # Count and print the number of currency images in the test set
    currency_images = test_generator.class_indices.get('currency', -1)
    if currency_images >= 0:
        print(f"Number of currency available is: {currency_images} going to Fine Grained Currency Recogniser")
    else:
        print("No currency images found in the test set.")


Found 311 images belonging to 3 classes.
Found 305 images belonging to 3 classes.
Epoch 1/10
10/10 [==============================] - 23s 2s/step - loss: 10.4974 - accuracy: 0.8103 - val_loss: 9.2425 - val_accuracy: 0.9836
Epoch 2/10
10/10 [==============================] - 21s 2s/step - loss: 8.5023 - accuracy: 0.9871 - val_loss: 7.7033 - val_accuracy: 0.9869
Epoch 3/10
10/10 [==============================] - 20s 2s/step - loss: 6.9975 - accuracy: 0.9968 - val_loss: 6.3071 - val_accuracy: 0.9869
Epoch 4/10
10/10 [==============================] - 21s 2s/step - loss: 5.6990 - accuracy: 1.0000 - val_loss: 5.1458 - val_accuracy: 0.9869
Epoch 5/10
10/10 [==============================] - 20s 2s/step - loss: 4.6683 - accuracy: 0.9968 - val_loss: 4.2150 - val_accuracy: 0.9869
Epoch 6/10
10/10 [==============================] - 21s 2s/step - loss: 3.8186 - accuracy: 0.9936 - val_loss: 3.4672 - val_accuracy: 0.9869
Epoch 7/10
10/10 [==============================] - 21s 2s/step - loss: 3.152